In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

## Partition base

In [2]:
dt=pd.read_excel(r"entreprises_france_2022.xlsx")

In [3]:
df=dt.iloc[1:5,:]

## Scraping 

In [4]:
url="https://entreprises.lefigaro.fr/camping-de-nogarede-66/entreprise-910468644"

In [5]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")

# Initialize WebDriver with auto driver management
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), 
    options=chrome_options
)

In [6]:
driver.get(url)

In [7]:
a=df["Siren"]

In [8]:
a

1    910482355
2    910500693
3    910499896
4    910468644
Name: Siren, dtype: int64

In [9]:
siren_list = []
for i in a:
    siren_list.append(i)

In [10]:
mandataire_list=[] 
activite_list=[] 

In [11]:
def process(text):
    t=0
    text=text.split('\n')
    for i in text:
        if 'Mandataires sociaux' in i:
            t=1
            mandataire_list.append(i)
            print('existe mandataire')
    if t==0:
        mandataire_list.append('introuvable')

In [12]:
def process1(text):
    t=0
    text=text.split('\n')
    for i in text:
        if 'Activité :' in i:
            t=1
            activite_list.append(i)
            print('existe activite')
    if t==0:
        activite_list.append('introuvable')

In [13]:
for i in siren_list:
    try:
        searchbox = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/header/div[2]/div[1]/div/nav/div[4]/div/form/fieldset/input[1]"))
        )
        searchbox.send_keys(i)
        time.sleep(10)

        l = driver.find_element(By.XPATH, "/html/body/div[2]/div/header/div[2]/div[1]/div/nav/div[4]/div/form/fieldset/input[2]")
        driver.execute_script("arguments[0].click();", l)
        
        try:
            table = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div[1]/div/main/div/div/div[5]/div[2]")
            table_data = table.text.split('\n')

            mandataires = []
            activites = []

            for line in table_data:
                if 'Mandataires sociaux' in line:
                    mandataires.append(line.strip())
                elif 'Activité :' in line:
                    activites.append(line.strip())

            if mandataires:
                mandataire_list.append(", ".join(mandataires))
            else:
                mandataire_list.append('introuvable')

            if activites:
                activite_list.append(", ".join(activites)) 
            else:
                activite_list.append('introuvable')
        
        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour {i}: {str(e)}")
            mandataire_list.append('introuvable')
            activite_list.append('introuvable')

    except Exception as e:
        print(f"Erreur lors de la recherche pour {i}: {str(e)}")
        mandataire_list.append('introuvable')
        activite_list.append('introuvable')

    driver.get(url)

Erreur lors de l'extraction des données pour 910482355: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div/div[2]/div[1]/div/main/div/div/div[5]/div[2]"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF78E7E2775+28773]
	(No symbol) [0x00007FF78E74AFB0]
	(No symbol) [0x00007FF78E5E552A]
	(No symbol) [0x00007FF78E638EAE]
	(No symbol) [0x00007FF78E63919C]
	(No symbol) [0x00007FF78E6823F7]
	(No symbol) [0x00007FF78E65EFDF]
	(No symbol) [0x00007FF78E67F1A4]
	(No symbol) [0x00007FF78E65ED43]
	(No symbol) [0x00007FF78E62A548]
	(No symbol) [0x00007FF78E62B6B1]
	GetHandleVerifier [0x00007FF78EB0F45D+3358029]
	GetHandleVerifier [0x00007FF78EB2430D+3443709]
	GetHandleVerifier [0x00007FF78EB183FD+3394797]
	GetHandleVerifier [0x00007FF78E8A929B+842635]
	(No symbol) [

In [14]:
mandataire_list

['introuvable',
 'Mandataires sociaux : Démission de M Jérôme SCHULER (Co-Gérant), confirmation de M Sebastien LECERF (Gérant), Mandataires sociaux : Nomination de M Sébastien LECERF (Gérant), nomination de M Jérôme SCHULER (Gérant), Mandataires sociaux : Gérant : LECERF Sébastien, Stéphane, André ; Gérant : SCHULER Jérôme, Pierre, Alexis',
 'Mandataires sociaux : Nomination de M Antonin Viaud (Président), nomination de M Clement Viaud (Directeur Général), Mandataires sociaux : Président : Viaud, Antonin, Directeur général : Viaud, Clement',
 'Mandataires sociaux : Nomination de M Patrice COMBRES (Gérant)']

In [15]:
activite_list

['introuvable',
 "Activité : La production et la vente de bières artisanales et généralement de toutes boissons. La location de tireuses à bières et de matériels liés à l'activité. Vente de produits dérivés et épicerie sèche. L'organisation d'événements autour de l'activité et d'ateliers. Plus généralement toute activité connexe et complémentaire, Activité : la production et la vente de bières artisanales et généralement de toute boissons ; la location de tireuses et matériels liés à l'activité ; la vente de produits dérivés et épicerie sèche ; l'organisation d'évènements autour de l'activité et d'ateliers ; et de toute activité connexe et complémentaire.",
 "Activité : - La conception, le développement, la fabrication, la distribution, la commercialisation de pièces détachées et d'accessoires de cycles. - Les activités de bureau d'études. - La prise, l'acquisition, l'exploitation ou la cession de toutes marques, de tous procédés, brevets et droits de propriété intellectuelle concernan

In [16]:
c1=pd.DataFrame(mandataire_list,columns = ['Mandataire'])

In [17]:
c2=pd.DataFrame(activite_list,columns = ['Description activité'])

In [18]:
c1.to_excel(r'mandataire_p1.xlsx', index = False)

In [19]:
c2.to_excel(r'activite_p1.xlsx', index = False)